In [1]:
import scanpy as sc
from cellflow.model import CellFlow

c:\Users\zhixu\AppData\Local\Programs\Python\Python312\Lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
c:\Users\zhixu\AppData\Local\Programs\Python\Python312\Lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
c:\Users\zhixu\AppData\Local\Programs\Python\Python312\Lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
c:\Users\zhixu\AppData\Local\Programs\Python\Python312\Lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
c:\Users\zhixu\AppData\Local\Programs\Python\Python312\Lib\site-packag

In [ ]:
filePath = "data/vcc_data/adata_Training.h5ad"
# filePath = "vcc_sample.h5ad"

In [3]:
adata = sc.read_h5ad(filePath)

In [4]:
adata

AnnData object with n_obs × n_vars = 88509 × 18080
    obs: 'target_gene', 'guide_id', 'batch', 'control'
    var: 'gene_id'

In [5]:
adata.obs['control'] = [(lambda x: True if x == "non-targeting" else False)(x) for x in adata.obs['target_gene']]

In [6]:
sample = sc.pp.subsample(adata, fraction=0.4, copy = True)

In [9]:
sc.write("vcc_sample.h5ad", sample)

In [6]:
cf = CellFlow(adata)

In [ ]:
# Prepare gene embeddings
# Sort out target genes
genes = adata.obs[adata.obs['control'] == False]['target_gene'].to_list()


In [ ]:
# Parameters for preparing data
sample_rep = "X"
control_key = "control"
perturbation_covariates = {"gene": "target_gene"}
split_covariates = ["batch"]
perturbation_covariate_reps = {"gene": "gene_embeddings"}

In [98]:
import requests
from UniProtMapper import ProtMapper
def get_protein_sequence_by_gene(gene_name):
    mapper = ProtMapper()
    result, failed = mapper.get(
        ids=gene_name, from_db="Gene_Name", to_db="UniProtKB"
    )
    result = result[(result['Organism'] == "Homo sapiens (Human)")&(result['Reviewed'] == "reviewed")]
    protein = result.iloc[0]["Entry"]
    # print(protein)
    # Define the UniProt API endpoint
    sequence_url = f"https://www.uniprot.org/uniprot/{protein}.fasta"
    sequence_response = requests.get(sequence_url)
        
    if sequence_response.status_code == 200:
        # Extract and return the protein sequence
        sequence = ''.join(sequence_response.text.splitlines()[1:])
        return sequence
    else:
        return "NONE"

# Example usage
gene_name = "PTPRC"  # Replace with your gene name
protein_sequence = get_protein_sequence_by_gene(gene_name)
print(f"Protein Sequence for {gene_name}:\n{protein_sequence}")

Fetched: 500 / 1299
    From   Entry   Entry Name  Reviewed  \
2  PTPRC  P08575  PTPRC_HUMAN  reviewed   

                                       Protein names  Gene Names  \
2  Receptor-type tyrosine-protein phosphatase C (...  PTPRC CD45   

               Organism Length  
2  Homo sapiens (Human)   1306  
P08575
Protein Sequence for PTPRC:
MTMYLWLKLLAFGFAFLDTEVFVTGQSPTPSPTGLTTAKMPSVPLSSDPLPTHTTAFSPASTFERENDFSETTTSLSPDNTSTQVSPDSLDNASAFNTTGVSSVQTPHLPTHADSQTPSAGTDTQTFSGSAANAKLNPTPGSNAISDVPGERSTASTFPTDPVSPLTTTLSLAHHSSAALPARTSNTTITANTSDAYLNASETTTLSPSGSAVISTTTIATTPSKPTCDEKYANITVDYLYNKETKLFTAKLNVNENVECGNNTCTNNEVHNLTECKNASVSISHNSCTAPDKTLILDVPPGVEKFQLHDCTQVEKADTTICLKWKNIETFTCDTQNITYRFQCGNMIFDNKEIKLENLEPEHEYKCDSEILYNNHKFTNASKIIKTDFGSPGEPQIIFCRSEAAHQGVITWNPPQRSFHNFTLCYIKETEKDCLNLDKNLIKYDLQNLKPYTKYVLSLHAYIIAKVQRNGSAAMCHFTTKSAPPSQVWNMTVSMTSDNSMHVKCRPPRDRNGPHERYHLEVEAGNTLVRNESHKNCDFRVKDLQYSTDYTFKAYFHNGDYPGEPFILHHSTSYNSKALIAFLAFLIIVTSIALLVVLYKIYDLHKKRSCNLDEQQELVERDDEKQLMNVEPIHADILLETYKRKIADEGRLFLA

In [78]:
import torch
from esm import pretrained
class ESMConverter:
  def __init__(self, model:str):
    self.model, self.alphabet = pretrained.load_model_and_alphabet(model)
    self.batch_converter = self.alphabet.get_batch_converter()

  def convert(self, sequences):
    batch_labels, batch_strs, batch_tokens = self.batch_converter(sequences)
    with torch.no_grad():
      token_embeddings = self.model(batch_tokens, repr_layers=[33])
      embeddings = token_embeddings['representations'][33]
      average_embeddings = embeddings.mean(dim=1)
    return average_embeddings

In [83]:
converter = ESMConverter("esm2_t33_650M_UR50D")
sequences = [(gene_name, protein_sequence), (gene_name, protein_sequence)]
embedding = converter.convert(sequences)

In [85]:
embedding.tolist()

[[0.017376022413372993,
  -0.049582865089178085,
  0.001769163296557963,
  0.07278474420309067,
  -0.02806677483022213,
  -0.0544624850153923,
  0.08051512390375137,
  -0.025202667340636253,
  0.045179855078458786,
  0.04827535152435303,
  0.013529905118048191,
  -0.0010761143639683723,
  0.0856604352593422,
  0.02682577632367611,
  0.014227298088371754,
  0.10517655313014984,
  0.02552144229412079,
  0.070123091340065,
  -0.03833439201116562,
  0.06838962435722351,
  0.023443855345249176,
  -0.0017306683585047722,
  0.08931249380111694,
  0.02120235562324524,
  0.10267539322376251,
  0.058262258768081665,
  0.03473470360040665,
  -0.005682071670889854,
  -0.04610012471675873,
  -0.07666579633951187,
  0.0464070625603199,
  0.03289928659796715,
  0.052423954010009766,
  -0.03592976927757263,
  0.005688492674380541,
  0.07507090270519257,
  -0.046404093503952026,
  -0.001078522065654397,
  -0.0038337816949933767,
  -0.014473299495875835,
  -0.05854271352291107,
  -0.02697961963713169,
 

In [40]:
from UniProtMapper import ProtMapper

In [58]:
mapper = ProtMapper()

result, failed = mapper.get(
    ids=["TP53"], from_db="Gene_Name", to_db="UniProtKB"
)

Fetched: 500 / 734


In [70]:
result[(result['Reviewed'] == "reviewed") & (result['Organism'] == "Homo sapiens (Human)")]

,From,Entry,Entry Name,Reviewed,Protein names,Gene Names,Organism,Length
6,TP53,P04637,P53_HUMAN,reviewed,Cellular tumor antigen p53 (Antigen NY-CO-13) ...,TP53 P53,Homo sapiens (Human),393
